In [ ]:
from numpy.random import binomial
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
interim = Path('../../data/interim')
events = pd.read_pickle(Path(interim) / 'events.pkl')

## Prep data by cleaning states and events

In [ ]:
# Limit to main event types and relevant variables for simplicity
states_data = events.loc[
    events.EVENT_CD.isin([2, 3, 14, 20, 21, 22, 23]), # 15, 16
    ['EVENT_CD', 'BASE1_RUN_ID', 'BASE2_RUN_ID', 
    'BASE3_RUN_ID', 'BAT_DEST_ID', 'RUN1_DEST_ID', 'RUN2_DEST_ID', 
    'RUN3_DEST_ID', 'OUTS_CT', 'EVENT_OUTS_CT', 'EVENT_RUNS_CT']
]

# Come up with starting base state for each event
states_data['1b'] = np.where(states_data['BASE1_RUN_ID'].isna(), 0, 1)
states_data['2b'] = np.where(states_data['BASE2_RUN_ID'].isna(), 0, 1)
states_data['3b'] = np.where(states_data['BASE3_RUN_ID'].isna(), 0, 1)
states_data['bases'] = states_data['1b'] + states_data['2b']*2 + states_data['3b']*4

# Come up with ending base state for each event
states_data['1b_new'] = np.where(
    (states_data['BAT_DEST_ID'] == 1) |
    (states_data['RUN1_DEST_ID'] == 1) |
    (states_data['RUN2_DEST_ID'] == 1) |
    (states_data['RUN3_DEST_ID'] == 1),
    1, 0
)

states_data['2b_new'] = np.where(
    (states_data['BAT_DEST_ID'] == 2) |
    (states_data['RUN1_DEST_ID'] == 2) |
    (states_data['RUN2_DEST_ID'] == 2) |
    (states_data['RUN3_DEST_ID'] == 2),
    1, 0
)

states_data['3b_new'] = np.where(
    (states_data['BAT_DEST_ID'] == 3) |
    (states_data['RUN1_DEST_ID'] == 3) |
    (states_data['RUN2_DEST_ID'] == 3) |
    (states_data['RUN3_DEST_ID'] == 3),
    1, 0
)

states_data['bases_new'] = states_data['1b_new'] + states_data['2b_new']*2 + states_data['3b_new']*4

# Clean up outs and events
states_data = states_data.rename(columns={'OUTS_CT': 'outs', 'EVENT_RUNS_CT': 'runs'})
states_data['outs_new'] = states_data['outs'] + states_data['EVENT_OUTS_CT']

# Only keep base variables
states = ['outs', 'bases', 'outs_new', 'bases_new']
states_data = states_data.loc[:, states + ['EVENT_CD', 'runs']]

states_data.head()

## New State Probabilites conditional on starting state and event

In [ ]:
new_state_prob = states_data.copy()
new_state_prob['freq'] = 1

# Calculate how often a state to state transition occurs for a given event, and associated number of runs
new_state_prob = new_state_prob.groupby(states + ['EVENT_CD']).agg({'runs':'mean', 'freq': 'sum'})
new_state_prob.runs = new_state_prob.runs.round(1)

# Only keep the transitions that are non-negligible for a given starting state and event
new_state_prob['totals'] = new_state_prob.groupby(['EVENT_CD', 'outs', 'bases'])['freq'].transform('sum')
new_state_prob['new_state_prob'] = new_state_prob['freq'] / new_state_prob['totals']
new_state_prob = new_state_prob.loc[new_state_prob['new_state_prob'] >= .05]
del new_state_prob['totals']
del new_state_prob['new_state_prob']

# Calculate probabilities of ending state given starting state and event
new_state_prob['totals'] = new_state_prob.groupby(['EVENT_CD', 'outs', 'bases'])['freq'].transform('sum')
new_state_prob['new_state_prob'] = new_state_prob['freq'] / new_state_prob['totals']
del new_state_prob['totals']
del new_state_prob['freq']

# Clean up index and ordering
new_state_prob = new_state_prob.reset_index()
new_state_prob = new_state_prob.sort_values(['outs', 'bases', 'EVENT_CD', 'outs_new', 'bases_new'])

new_state_prob.head(15)

## Calculate Event Odds Conditional on State

In [ ]:
event_prob = states_data.groupby(['outs', 'bases', 'EVENT_CD']).size().to_frame()
event_prob.columns = ['count']
event_prob['total'] = event_prob.groupby(['outs', 'bases'])['count'].transform('sum')
event_prob['event_prob'] = event_prob['count'] / event_prob['total']
event_prob = event_prob.drop(['count', 'total'], axis = 1)
event_prob = event_prob.reset_index()
event_prob.head(15)

# Calculate Transition Probabilities

In [ ]:
transition_prob = new_state_prob.merge(event_prob, on=['outs', 'bases', 'EVENT_CD']).set_index(states)
transition_prob['transition_prob'] = transition_prob['new_state_prob'] * transition_prob['event_prob']
transition_prob = transition_prob.groupby(states)['transition_prob'].sum().to_frame()

## Calculate Reward Matrix

In [ ]:
rewards = new_state_prob.groupby(states)['runs'].mean().to_frame()

## Set up matrices

In [ ]:
outs_mat = [0, 1, 2]
bases_mat = [0, 1, 2, 3, 4, 5, 6, 7]
mind = pd.MultiIndex.from_product(
    [outs_mat, bases_mat, outs_mat, bases_mat], 
    names = ['outs', 'bases', 'outs_new', 'bases_new']
)

In [ ]:
transition_prob = transition_prob.reindex(mind, fill_value=0)
transition_prob_wide = transition_prob.reset_index().pivot(
    index=['outs', 'bases'], 
    columns=['outs_new', 'bases_new'], 
    values='transition_prob'
)
P = transition_prob_wide.to_numpy()

In [ ]:
rewards = rewards.reindex(mind, fill_value=0)
rewards_wide = rewards.reset_index().pivot(
    index=['outs', 'bases'], 
    columns=['outs_new', 'bases_new'], 
    values='runs'
)
R = rewards_wide.to_numpy()

## Calculate Expected Runs

In [ ]:
I = np.identity(P.shape[0])

In [ ]:
Q = np.sum(R * P, axis = 1).reshape(24,1)

In [ ]:
v = np.linalg.solve((I-P), Q)

In [ ]:
v

## Compare with Actual Run Values

In [ ]:
events[events.EVENT_CD.isin([2, 3, 14, 20, 21, 22, 23])].EVENT_RUNS_CT.sum()

In [ ]:
events[events.EVENT_CD.isin([2, 3, 14, 20, 21, 22, 23])].groupby(['GAME_ID', 'BAT_HOME_ID', 'INN_CT']).size()

In [ ]:
print(1455709 / 3068406)